In [61]:
import plotly.graph_objects as go
import ipywidgets as widgets

In [62]:
initial_amount_max = 100000
monthly_contribution_max = 10000
interest_rate_max = 0.20
duration_max = 50

In [63]:
def interests_computation(initial_amount=0, monthly_contribution=200, interest_rate=0.05, duration=10):
  monthly_value = []
  total_contribution = []
  total_interest = []

  for i in range(duration):
    if i == 0:
      previous_m = initial_amount
      previous_c = initial_amount
      previous_i = 0
    else:
      previous_m = monthly_value[i-1]
      previous_c = total_contribution[i-1]
      previous_i = total_interest[i-1]

    monthly_value.append(int((previous_m+(monthly_contribution*12))*(1+interest_rate)))
    total_contribution.append(previous_c+(monthly_contribution*12))
    total_interest.append(previous_i + int((previous_m+(monthly_contribution*12))*interest_rate))

  X = list(range(1, duration+1))

  return X, monthly_value, total_contribution, total_interest

X, monthly_value, total_contribution, total_interest = interests_computation()

annotation = dict(
    x= 2,
    y= int(monthly_value[-1]*0.8),
    text= f'Total amount: {monthly_value[-1]}€ <br>Total contribution: {total_contribution[-1]}€ <br>Total interest: {total_interest[-1]}€',  
    showarrow= False
)

layout = go.Layout(
        xaxis = dict(title = 'Years'),
        yaxis = dict(title = '€'),
        title = f'Investment performance starting with {0}€, followed by {200}€/month at {int(0.05*100)}%/year for {10} years',
        annotations = [annotation])

trace1 = go.Scatter(x=X, y=total_contribution, fill='tozeroy', name="Total Contribution") # fill down to xaxis
trace2 = go.Scatter(x=X, y=monthly_value, fill='tonexty', name="Interests") # fill to trace0 y

fig = go.FigureWidget(data=[trace1, trace2], layout=layout)

  #print("Total Amount :", monthly_value[-1], "€")
  #print("Total Contribution :", total_contribution[-1], "€")
  #print("Total Interests : ", total_interest[-1], "€")

In [64]:
initial_amount = widgets.IntSlider(
    value=0.0,
    min=0.0,
    max=initial_amount_max,
    step=1000.0,
    description='Initial Amount:',
    continuous_update=False
)

monthly_contribution = widgets.IntSlider(
    value=200.0,
    min=0.0,
    max=monthly_contribution_max,
    step=100.0,
    description='Monthly Contrib:',
    continuous_update=False
)

interest_rate = widgets.FloatSlider(
    value=0.05,
    min=0.0,
    max=interest_rate_max,
    step=0.01,
    description='Interest Rate:',
    continuous_update=False
)

duration = widgets.IntSlider(
    value=10.0,
    min=1.0,
    max=duration_max,
    step=1.0,
    description='Duration:',
    continuous_update=False
)

def response(change):
  X, monthly_value, total_contribution, total_interest = interests_computation(initial_amount.value, monthly_contribution.value, interest_rate.value, duration.value)

  with fig.batch_update():
    fig.data[0].x = X
    fig.data[0].y = total_contribution
    fig.data[1].x = X
    fig.data[1].y = monthly_value
    fig.layout.xaxis.title = 'Years'
    fig.layout.yaxis.title = '€'
    fig.layout.title = f'Investment performance starting with {initial_amount.value}€, followed by {monthly_contribution.value}€/month at {int(interest_rate.value*100)}%/year for {duration.value} years'
    fig.layout.annotations[0].text = f'Total amount: {monthly_value[-1]}€ <br>Total contribution: {total_contribution[-1]}€ <br>Total interest: {total_interest[-1]}€'
    fig.layout.annotations[0].y = int(monthly_value[-1]*0.8)

container1 = widgets.HBox([initial_amount, duration])
container2 = widgets.HBox([monthly_contribution, interest_rate])

initial_amount.observe(response, names="value")
monthly_contribution.observe(response, names="value")
interest_rate.observe(response, names="value")
duration.observe(response, names="value")

widgets.VBox([container1,
              container2,
              fig])